In [54]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

In [55]:
#Importo
df_producto=pd.read_csv('../data/EXPO_DOC.csv', encoding='latin-1', dtype={'CUIT':'str',
                                                                                            'dia':'str',
                                                                                            'mes':'str',
                                                                                            'anio':'str'})

#DF con las fechas
inicio=df_producto.sort_values(['anio','mes'],ascending=True).reset_index(drop=True).mes[0]+'/01/'+df_producto.sort_values(['anio','mes'],ascending=True).reset_index(drop=True).anio[0]
mes=int(df_producto.sort_values(['anio','mes'],ascending=True).reset_index(drop=True).mes.iloc[-1])+1
fin=str(mes)+'/01/'+df_producto.sort_values(['anio','mes'],ascending=True).reset_index(drop=True).anio.iloc[-1]
monthDates = pd.DataFrame({
    'fecha': pd.date_range(start=inicio, end=fin, freq='M').strftime('%m-%Y')
})

# Correcciones de lectura
df_producto.pais_descri=df_producto.pais_descri.apply(lambda x: x.capitalize())
df_producto.empresa=df_producto.empresa.apply(lambda x: x.lower())

#Fob por tonelada importada
df_producto['fob_unitario_ton']=df_producto.fob/df_producto.pnet*1000

#Importo datos de los precios de referencia del banco mundial
bm=pd.read_excel('../data/CMO-Historical-Data-Monthly.xlsx',sheet_name='Monthly Prices',skiprows=6)

bm=(bm[[bm.columns[0],bm.columns[24],bm.columns[25],bm.columns[26],bm.columns[30],bm.columns[37]]]
    .rename(columns={'Unnamed: 0':'Fecha', 'SOYBEANS':'bm_soja',bm.columns[37]:'bm_trigo',bm.columns[30]:'bm_maiz', 
                     bm.columns[25]:'bm_aceitesoja', bm.columns[26]:'bm_harinasoja'}))
bm['anio']=bm.Fecha.apply(lambda x: x[:4])
bm['mes']=bm.Fecha.apply(lambda x: x[5:])
bm.drop('Fecha',axis=1,inplace=True)

# Separo las dataframes para cada producto, haciendolo mas facil de manipular
dfs_producto=[i for x, i in df_producto.groupby('NOMEN', as_index=False)]

def genero_df(df):
    df=df.sort_values(['anio','mes','dia'], ascending=True).reset_index(drop=True)
    df['fob_unitario_ton_capit']=df.fob_unitario_ton*df.fob_unitario_ton_ref.iloc[-1]/df.fob_unitario_ton_ref
    df['diferencia_ref']=df.fob_unitario_ton-df.fob_unitario_ton_ref
    df['diferencia_ref_capi']=df.fob_unitario_ton_capit-df.fob_unitario_ton_ref.iloc[-1]
    df['fecha']=df["mes"].astype(str)+'-'+df['anio'].astype(str)
    df['fecha']=pd.to_datetime(df["fecha"]).dt.strftime('%m-%Y')
    return df

trigo=pd.merge(left=dfs_producto[0],right=bm[['anio','mes','bm_trigo']], on=['anio','mes'],how='left').rename({'bm_trigo':'fob_unitario_ton_ref'},axis=1) #concat
trigo=genero_df(trigo)

maiz=pd.merge(left=dfs_producto[1],right=bm[['anio','mes','bm_maiz']], on=['anio','mes'],how='left').rename({'bm_maiz':'fob_unitario_ton_ref'},axis=1) #concat
maiz=genero_df(maiz)

soja=pd.merge(left=dfs_producto[2],right=bm[['anio','mes','bm_soja']], on=['anio','mes'],how='left').rename({'bm_soja':'fob_unitario_ton_ref'},axis=1) #concat
soja=genero_df(soja)

aceite=pd.merge(left=dfs_producto[3],right=bm[['anio','mes','bm_aceitesoja']], on=['anio','mes'],how='left').rename({'bm_aceitesoja':'fob_unitario_ton_ref'},axis=1) #concat
aceite=genero_df(aceite)

harina=pd.merge(left=dfs_producto[4],right=bm[['anio','mes','bm_harinasoja']], on=['anio','mes'],how='left').rename({'bm_harinasoja':'fob_unitario_ton_ref'},axis=1) #concat
harina=genero_df(harina)

#Biodisel no está en el BM, los p de referencia se determinan en base a los datos mensuales
biodisel_mensual=dfs_producto[5].groupby(['mes','anio'],as_index=False).sum().sort_values(['anio','mes'])
biodisel_mensual['fob_unitario_ton_ref']=biodisel_mensual.fob/biodisel_mensual.pnet*1000
biodisel=pd.merge(left=dfs_producto[5],right=biodisel_mensual[['anio','mes','fob_unitario_ton_ref']], on=['anio','mes'],how='left') #concat
biodisel=genero_df(biodisel)

desde=df_producto.sort_values(['anio','mes','dia'], ascending=True)['anio'][0]
hasta=df_producto.sort_values(['anio','mes','dia'], ascending=True)['anio'].iloc[-1]

In [56]:
#Funciones
def outliers(df):
    outliers=df.sort_values('fob_unitario_ton',ascending=False)
    return outliers[['anio','dia','mes','NOMEN','CUIT','empresa','pais_descri','fob','pnet','fob_unitario_ton','fob_unitario_ton_ref']]

def elimina_outliers(df,fob_unit_max): 
    return df[df.fob_unitario_ton<fob_unit_max].reset_index(drop=True)

n_paises=int(input('cuantos paises: '))

def get_top_paises(df, n_paises=n_paises):
    top_df=df.groupby(['pais','pais_descri'],as_index=False).sum().sort_values('fob',ascending=False).pais_descri.unique()[:n_paises]
    return df[df.pais_descri.isin(top_df)]


# Trigo
trigo_outliers=outliers(trigo) #Para observar outliers
trigo=elimina_outliers(trigo, 1000) #Precios de 1000 usd para abajo
df_top_trigo=get_top_paises(trigo,n_paises) #Top paises, para evitar colapso de los colores

# Maiz
maiz_outliers=outliers(maiz)
maiz=elimina_outliers(maiz,1000)
df_top_maiz=get_top_paises(maiz,n_paises)

# Soja
soja_outliers=outliers(soja)
soja=elimina_outliers(soja,1000)
df_top_soja=get_top_paises(soja,n_paises)

# Biodisel
biodisel_outliers=outliers(biodisel)
biodisel=elimina_outliers(biodisel,2000)
df_top_biodisel=get_top_paises(biodisel,n_paises)

# Aceite de soja: no hay outliers
aceite_outliers=outliers(biodisel)
aceite=elimina_outliers(aceite,2500)
df_top_aceite=get_top_paises(aceite,n_paises)

# Harina de soja: creo que no hay outliers
harina_outliers=outliers(harina)
# harina=elimina_outliers(harina,2000)
df_top_harina=get_top_paises(harina,n_paises)

In [57]:
def precio_boxplot_mes(producto):
     '''Producto='trigo','soja','maiz, biodisel, aceite soja, harina soja'''
     if producto=='soja':
          df=soja
          title_text=f'FOB por tonelada exportada de {producto} <br> <sup> Precio de referencia del Banco Mundial'
     elif producto=='trigo':
          df=trigo
          title_text=f'FOB por tonelada exportada de {producto} <br> <sup> Precio de referencia del Banco Mundial'
     elif producto=='maiz':
          df=maiz
          title_text=f'FOB por tonelada exportada de {producto} <br> <sup> Precio de referencia del Banco Mundial'
     elif producto=='biodisel':
          df=biodisel
          title_text=f'FOB por tonelada exportada de {producto} <br> <sup> Precio de referencia a partir de datos agrupados mensualmente'
     elif producto=='aceite soja':
          df=aceite
          title_text=f'FOB por tonelada exportada de {producto} <br> <sup> Precio de referencia del Banco Mundial'
     elif producto=='harina soja':
          df=harina
          title_text=f'FOB por tonelada exportada de {producto} <br> <sup> Precio de referencia del Banco Mundial'
     
     precio_soja_boxplot=px.box(
     df.round(1),
     x='fecha',
     y='fob_unitario_ton',
     color='fecha',
     labels={
          "pais_descri": "Destino",
          'fob_unitario_ton': 'Precio',
          'fob':'Fob',
          'pnet':'Kg',
          'fecha':'Fecha',
          'fob_unitario_ton_ref': 'Precio de referencia',
          'diferencia_ref':'Spread',
          'empresa':'Exportador'
          },
     hover_data={'pais_descri',
                    'fob_unitario_ton',
                    'fob_unitario_ton_ref',
                    'diferencia_ref',
                    'empresa',
                    'fob',
                    'pnet'        
                                   }
     )

     precio_soja_boxplot.update_yaxes(title_text= 'Fob unitario', 
                                   range=[df['fob_unitario_ton'].min()-100,df['fob_unitario_ton'].max()+100]
                                   )
     precio_soja_boxplot.update_xaxes(title_text='')

     precio_soja_boxplot.update_layout(separators=',.', font_family='Georgia', font_size=13,
                                   height=700, width=900,
                                   template = 'none',
                                   title_text=title_text,
                                   showlegend=False)

     return precio_soja_boxplot

def precio_boxplot_capitalizado(producto):
     '''Producto='trigo','soja','maiz, biodisel, aceite soja, harina soja'''
     if producto=='soja':
          df=soja
          title_text=f'FOB por tonelada exportada de {producto} {desde}-{hasta} <br> <sup> Capitalizado al último precio disponible. Precio de referencia del Banco Mundial'
     elif producto=='trigo':
          df=trigo
          title_text=f'FOB por tonelada exportada de {producto} {desde}-{hasta} <br> <sup> Capitalizado al último precio disponible. Precio de referencia del Banco Mundial'
     elif producto=='maiz':
          df=maiz
          title_text=f'FOB por tonelada exportada de {producto} {desde}-{hasta} <br> <sup> Capitalizado al último precio disponible. Precio de referencia del Banco Mundial'
     elif producto=='biodisel':
          df=biodisel
          title_text=f'FOB por tonelada exportada de {producto} {desde}-{hasta} <br> <sup> Capitalizado al último precio disponible. Precio de referencia a partir de datos agrupados mensualmente'
     elif producto=='aceite soja':
          df=aceite
          title_text=f'FOB por tonelada exportada de {producto} {desde}-{hasta} <br> <sup> Capitalizado al último precio disponible. Precio de referencia del Banco Mundial'
     elif producto=='harina soja':
          df=harina
          title_text=f'FOB por tonelada exportada de {producto} {desde}-{hasta} <br> <sup> Capitalizado al último precio disponible. Precio de referencia del Banco Mundial'
     
     precio_soja_boxplot=px.box(
     df.round(1),
     # x='fecha',
     x='fob_unitario_ton_capit',
     # color='fecha',
     labels={
          "pais_descri": "Destino",
          'fob_unitario_ton': 'Precio',
          'fob_unitario_ton_capit':'Precio capitalizado',
          'fob':'Fob',
          'pnet':'Kg',
          'fecha':'Fecha',
          'fob_unitario_ton_ref': 'Precio de referencia',
          'diferencia_ref':'Spread',
          'empresa':'Exportador'
          },
     hover_data={'pais_descri',
                 'fecha',
                    'fob_unitario_ton',
                    'fob_unitario_ton_capit',
                    'fob_unitario_ton_ref',
                    'diferencia_ref',
                    'empresa',
                    'fob',
                    'pnet'        
                                   }
     )

     precio_soja_boxplot.update_yaxes(title_text= '', 
                                   # range=[producto['fob_unitario_ton_capit'].min()-100,producto['fob_unitario_ton_capit'].max()+100]
                                   )
     precio_soja_boxplot.update_xaxes(title_text='Fob unitario')

     precio_soja_boxplot.update_layout(separators=',.', font_family='Georgia', font_size=13,
                                   height=400, width=700,
                                   template = 'none',
                                   title_text=title_text,
                                   showlegend=False)
     
     precio_soja_boxplot.add_vline(x=df.fob_unitario_ton_ref.iloc[-1],line_dash="dash",line_color="blue")

     return precio_soja_boxplot

def precio_boxplot(producto):
     '''Producto='trigo','soja','maiz, biodisel, aceite soja, harina soja'''
     if producto=='soja':
          df=soja
          title_text=f'FOB por tonelada exportada de {producto} {desde}-{hasta} <br> <sup> Capitalizado al último precio disponible. Precio de referencia del Banco Mundial'
     elif producto=='trigo':
          df=trigo
          title_text=f'FOB por tonelada exportada de {producto} {desde}-{hasta} <br> <sup> Capitalizado al último precio disponible. Precio de referencia del Banco Mundial'
     elif producto=='maiz':
          df=maiz
          title_text=f'FOB por tonelada exportada de {producto} {desde}-{hasta} <br> <sup> Capitalizado al último precio disponible. Precio de referencia del Banco Mundial'
     elif producto=='biodisel':
          df=biodisel
          title_text=f'FOB por tonelada exportada de {producto} {desde}-{hasta} <br> <sup> Capitalizado al último precio disponible. Precio de referencia a partir de datos agrupados mensualmente'
     elif producto=='aceite soja':
          df=aceite
          title_text=f'FOB por tonelada exportada de {producto} {desde}-{hasta} <br> <sup> Capitalizado al último precio disponible. Precio de referencia del Banco Mundial'
     elif producto=='harina soja':
          df=harina
          title_text=f'FOB por tonelada exportada de {producto} {desde}-{hasta} <br> <sup> Capitalizado al último precio disponible. Precio de referencia del Banco Mundial'
          
     precio_soja_boxplot=px.box(
     df.round(1),
     # x='fecha',
     x='fob_unitario_ton',
     # color='fecha',
     labels={
          "pais_descri": "Destino",
          'fob_unitario_ton': 'Precio',
          'fob':'Fob',
          'pnet':'Kg',
          'fecha':'Fecha',
          'fob_unitario_ton_ref': 'Precio de referencia',
          'diferencia_ref':'Spread',
          'empresa':'Exportador'
          },
     hover_data={'pais_descri',
                    'fob_unitario_ton',
                    'fob_unitario_ton_ref',
                    'diferencia_ref',
                    'empresa',
                    'fob',
                    'pnet'        
                                   }
     )

     precio_soja_boxplot.update_yaxes(title_text= '', 
                                   # range=[df['fob_unitario_ton'].min()-100,df['fob_unitario_ton'].max()+100]
                                   )
     precio_soja_boxplot.update_xaxes(title_text='Fob unitario')

     precio_soja_boxplot.update_layout(separators=',.', font_family='Georgia', font_size=13,
                                   height=400, width=700,
                                   template = 'none',
                                   title_text=title_text,
                                   showlegend=False)

     return precio_soja_boxplot

def plot_precio(y, color, producto, pais='nan'):
     '''y='fob_unitario_ton', 'diferencia_ref'
     color='pais_descri','empresa'     
     Producto='trigo','soja','maiz
     Pais='Estados unidos'''
     if pais != 'nan':
          if producto=='soja':
               df=soja[soja['pais_descri']==pais.capitalize()].sort_values(['anio','mes','dia'],ascending=True).reset_index(drop=True)
          elif producto=='trigo':
               df=trigo[trigo['pais_descri']==pais.capitalize()].sort_values(['anio','mes','dia'],ascending=True).reset_index(drop=True)
          elif producto=='maiz':
               df=maiz[maiz['pais_descri']==pais.capitalize()].sort_values(['anio','mes','dia'],ascending=True).reset_index(drop=True)
          if producto=='biodisel':
               df=biodisel[biodisel['pais_descri']==pais.capitalize()].sort_values(['anio','mes','dia'],ascending=True).reset_index(drop=True)
          if producto=='aceite soja':
               df=aceite[aceite['pais_descri']==pais.capitalize()].sort_values(['anio','mes','dia'],ascending=True).reset_index(drop=True)
          if producto=='harina soja':
               df=harina[harina['pais_descri']==pais.capitalize()].sort_values(['anio','mes','dia'],ascending=True).reset_index(drop=True)
     else:
          if producto=='soja':
               df=df_top_soja
          elif producto=='trigo':
               df=df_top_trigo
          elif producto=='maiz':
               df=df_top_maiz
          elif producto=='biodisel':
               df=df_top_biodisel
          elif producto=='aceite soja':
               df=df_top_aceite
          elif producto=='harina soja':
               df=df_top_harina
          
     
     if y=='fob_unitario_ton' and pais == 'nan':
          title_1= 'Fob unitario'
          title_2= f'FOB por tonelada exportada de {producto} <br> <sup> Precio de referencia del Banco Mundial o a partir de datos mensuales'
     elif y=='diferencia_ref' and pais == 'nan':
          title_1= 'Diferencia en USD'
          title_2= f'Diferencia con precio de referencia en USD por tonelada exportada de {producto} <br> <sup> Precio de referencia del Banco Mundial o a partir de datos mensuales'
     elif y=='fob_unitario_ton' and pais != 'nan':
          title_1= 'Fob unitario'
          title_2= f'FOB por tonelada exportada de {producto} <br> <sup> Destino: {pais.capitalize()}. Precio de referencia del Banco Mundial o a partir de datos mensuales'
     elif y=='diferencia_ref' and pais != 'nan':
          title_1= 'Diferencia en USD'
          title_2= f'Diferencia con precio de referencia en USD por tonelada exportada de {producto} <br> <sup> Destino: {pais.capitalize()}. Precio de referencia del Banco Mundial o a partir de datos mensuales'

          
     precio_soja_plot=px.scatter(
     df.round(1),
     x='fecha',
     y=y,
     color=color,
     category_orders={"fecha": monthDates.fecha},
     labels={
          "pais_descri": "Destino",
          'fob_unitario_ton': 'Precio',
          'fob':'Fob',
          'pnet':'Kg',
          'fecha':'Fecha',
          'fob_unitario_ton_ref': 'Precio de referencia',
          'diferencia_ref':'Spread',
          'empresa':'Exportador'
          },
     hover_data={'pais_descri',
                    'fob_unitario_ton',
                    'fob_unitario_ton_ref',
                    'diferencia_ref',
                    'empresa',
                    'fob',
                    'pnet'        
                                   }
     )

     precio_soja_plot.update_yaxes(title_text= title_1, 
                                   range=[df[y].min()-100,df[y].max()+100]
                                   )
     precio_soja_plot.update_xaxes(title_text='',type='category')

     precio_soja_plot.update_traces(marker=dict(size=12,
                                   line=dict(width=2,
                                             color='DarkSlateGrey')),
                    selector=dict(mode='markers'))

     precio_soja_plot.update_layout(separators=',.', font_family='Georgia', font_size=13,
                                   height=700, width=900,
                                   template = 'none',
                                   title_text=title_2)

     return precio_soja_plot


In [58]:
plot_precio(y='diferencia_ref',
            color='pais_descri',
            producto='soja',
            # pais='chile'
            )
# precio_boxplot_capitalizado('biodisel')


### Tabla 1
Los valores son a precios capitalizados

In [59]:
def paises_interesantes(producto):
    '''Producto='trigo','soja','maiz'''
    if producto=='soja':
        df=soja
    elif producto=='trigo':
        df=trigo
    elif producto=='maiz':
        df=maiz
        
    contar_paises=df.groupby('pais_descri').size().sort_values(ascending=False)
    paises=pd.DataFrame(contar_paises).reset_index().pais_descri.unique()[:9]
    return paises

paises_interesantes(producto='soja')

array(['Chile', 'Estados unidos', 'China', 'Uruguay', 'Brasil', 'Canada',
       'Paraguay', 'Egipto', 'Taiwan'], dtype=object)

In [60]:
def datos_tabla_paises(pais, producto):
    '''Producto='trigo','soja','maiz'''
    if producto=='soja':
        df=soja
    elif producto=='trigo':
        df=trigo
    elif producto=='maiz':
        df=maiz
    return [len(df[df.pais_descri==pais]),round(df[df.pais_descri==pais].fob_unitario_ton_capit.min(),1), round(df[df.pais_descri==pais].fob_unitario_ton_capit.quantile(0.25),1),round(df[df.pais_descri==pais].fob_unitario_ton_capit.quantile(0.5),1),round(df[df.pais_descri==pais].fob_unitario_ton_capit.quantile(0.75),1),round(df[df.pais_descri==pais].fob_unitario_ton_capit.max(),1)]

# A precios capitalizados

def tabla_1(producto):
    '''Producto='trigo','soja','maiz'''
    if producto=='soja':
        df=soja
    elif producto=='trigo':
        df=trigo
    elif producto=='maiz':
        df=maiz
        
    tabla_1_expo=pd.DataFrame({
        '':['No. of trade records','Lowest price','Lower quartile price','Median price','Upper quartile price','Highest price'],
        'Mundo':[len(df),round(df.fob_unitario_ton_capit.min(),1), round(df.fob_unitario_ton_capit.quantile(0.25),1),round(df.fob_unitario_ton_capit.quantile(0.5),1),round(df.fob_unitario_ton_capit.quantile(0.75),1),round(df.fob_unitario_ton_capit.max(),1)],
        # paises_interesantes[0]:datos_tabla_paises(paises_interesantes[0]),
        **{pais_interesante: datos_tabla_paises(pais_interesante, producto) for pais_interesante in paises_interesantes(producto)}
        })
    tabla_1_expo=tabla_1_expo.transpose().reset_index()
    header=tabla_1_expo.iloc[0]
    tabla_1_expo=tabla_1_expo[1:]
    tabla_1_expo.columns=header
    tabla_1_expo['No. of trade records']=tabla_1_expo['No. of trade records'].astype(int)
    tabla_1_expo.rename(columns={'':'Paises'},inplace=True)
    return tabla_1_expo



### Tabla de las posibles falsificaciones

In [61]:
def tabla_posibles_falsificaciones(producto):
    '''Producto='trigo','soja','maiz'''
    if producto=='soja':
        df=soja
    elif producto=='trigo':
        df=trigo
    elif producto=='maiz':
        df=maiz
    
    posibles_falsificaciones=df[(df.fob_unitario_ton_capit<df.fob_unitario_ton_capit.quantile(.25)) |
                (df.fob_unitario_ton_capit>df.fob_unitario_ton_capit.quantile(.75))]
    # posibles_falsificaciones.drop('index',axis=1,inplace=)
    return posibles_falsificaciones

# Exportar

In [62]:
plot_precio(y='diferencia_ref',color='pais_descri',producto='soja').write_html('../output/diferencia con ref soja pais.html')
plot_precio(y='fob_unitario_ton',color='pais_descri',producto='soja').write_html('../output/fob unitario por ton soja pais.html')
plot_precio(y='diferencia_ref',color='pais_descri',producto='trigo').write_html('../output/diferencia con ref trigo pais.html')
plot_precio(y='diferencia_ref',color='pais_descri',producto='maiz').write_html('../output/diferencia con ref maiz pais.html')
plot_precio(y='diferencia_ref',color='pais_descri',producto='biodisel').write_html('../output/diferencia con ref biodisel pais.html')
plot_precio(y='diferencia_ref',color='pais_descri',producto='aceite soja').write_html('../output/diferencia con ref aceite pais.html')
plot_precio(y='diferencia_ref',color='pais_descri',producto='harina soja').write_html('../output/diferencia con ref harina pais.html')


plot_precio(y='diferencia_ref',color='empresa',producto='soja',pais='estados unidos').write_html('../output/expo soja eeuu empresa.html')
precio_boxplot('soja').write_html('../output/Expo soja boxplot sin capitalizar.html')
precio_boxplot_capitalizado('trigo').write_html('../output/Expo trigo boxplot capitalizado.html')
precio_boxplot_capitalizado('maiz').write_html('../output/Expo maiz boxplot capitalizado.html')
precio_boxplot_capitalizado('soja').write_html('../output/Expo soja boxplot capitalizado.html')
precio_boxplot_capitalizado('biodisel').write_html('../output/Expo biodisel boxplot capitalizado.html')
precio_boxplot_capitalizado('aceite soja').write_html('../output/Expo aceite boxplot capitalizado.html')
precio_boxplot_capitalizado('harina soja').write_html('../output/Expo harina boxplot capitalizado.html')

In [63]:
writer = pd.ExcelWriter(f'../output/exportaciones soja trigo maiz.xlsx', engine='xlsxwriter')
tabla_1('soja').to_excel(writer, sheet_name='soja', index=False)
tabla_1('trigo').to_excel(writer, sheet_name='trigo', index=False)
tabla_1('maiz').to_excel(writer, sheet_name='maiz', index=False)
writer.save()

In [64]:
writer = pd.ExcelWriter(f'../output/exportaciones soja.xlsx', engine='xlsxwriter')
tabla_1('soja').to_excel(writer, sheet_name='tabla_1_expo', index=False)
soja.to_excel(writer, sheet_name='desagregado', index=False)
soja_outliers.to_excel(writer, sheet_name='soja outliers', index=False)
trigo_outliers.to_excel(writer, sheet_name='trigo outliers', index=False)
maiz_outliers.to_excel(writer, sheet_name='maiz outliers', index=False)
biodisel_outliers.to_excel(writer, sheet_name='biodisel outliers', index=False)
aceite_outliers.to_excel(writer, sheet_name='aceite outliers', index=False)
harina_outliers.to_excel(writer, sheet_name='harina outliers', index=False)
tabla_posibles_falsificaciones('soja').to_excel(writer, sheet_name='posibles_falsificaciones')
writer.save()